In [ ]:
!pip install transformers==4.9.2
!pip install nltk
!pip install datasets==1.14.0
!pip install sentencepiece
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

# play text2cypher

In [ ]:
import torch

In [ ]:
def text2cypher(text, model, tokenizer):
    #model
    print("=====❓Request=====")
    print(text)
    tokenized_txt = tokenizer([text], max_length=1024, padding="max_length", truncation=True)
    pred = tokenizer.batch_decode(
        model.generate(
            torch.LongTensor(tokenized_txt.data['input_ids']),
            torch.LongTensor(tokenized_txt.data['attention_mask']),
            num_beams=1, 
            max_length=256
            ), 
        skip_special_tokens=True 
    ) # More details see utils/dataset.py and utils/trainer.py
    print("=====💡Answer=====")
    print(pred)
    # text2cypher = 'MATCH (singer:`concert_singer.singer`) RETURN count(*)'
    return pred

In [14]:
from transformers import (
        AutoTokenizer, 
        T5ForConditionalGeneration)
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5-base", use_fast=False)
model = T5ForConditionalGeneration.from_pretrained("/home/22842219/Desktop/phd/SemanticParser4Graph/application/rel_db2kg/text2sql2cypher/pretrain")


file /home/22842219/Desktop/phd/SemanticParser4Graph/application/rel_db2kg/text2sql2cypher/pretrain/config.json not found


OSError: Can't load config for '/home/22842219/Desktop/phd/SemanticParser4Graph/application/rel_db2kg/text2sql2cypher/pretrain'. Make sure that:

- '/home/22842219/Desktop/phd/SemanticParser4Graph/application/rel_db2kg/text2sql2cypher/pretrain' is a correct model identifier listed on 'https://huggingface.co/models'

- or '/home/22842219/Desktop/phd/SemanticParser4Graph/application/rel_db2kg/text2sql2cypher/pretrain' is the correct path to a directory containing a config.json file



In [11]:
pred = text2cypher('how many singers do we have?', model, tokenizer)

=====❓Request=====
how many singers do we have?


TypeError: generate() got multiple values for argument 'max_length'

# play

In [ ]:
import sys
sys.path.append('/UnifiedSKGG-CYPHER')

In [ ]:
import os
import time
import torch
import datasets
from transformers import (
    HfArgumentParser,
    set_seed,
    AutoTokenizer
)
from utils.configue import Configure
from utils.training_arguments import WrappedSeq2SeqTrainingArguments

from filelock import FileLock
import nltk
with FileLock(".lock") as lock:
    nltk.download("punkt", quiet=True)
    nltk.download("stopwords", quiet=True)

In [ ]:
def play(txt, model, tokenizer):
  print("=====❓Request=====")
  print(txt)
  tokenized_txt = tokenizer([txt], max_length=1024, padding="max_length", truncation=True)
  pred = tokenizer.batch_decode(
      model.generate(
        torch.LongTensor(tokenized_txt.data['input_ids']),
        torch.LongTensor(tokenized_txt.data['attention_mask']),
        num_beams=1, 
        max_length=256
        ), 
      skip_special_tokens=True 
  ) # More details see utils/dataset.py and utils/trainer.py
  print("=====💡Answer=====")
  print(pred)
  return pred

In [ ]:
#  Spider (with cell value)

# Set args here for runnning on notebook, we make them out here to make it more illustrative.
sys.argv = ['/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py', # This is the name of your .py launcher when you run this line of code.
            # belows are the parameters we set, take spider for example
            '--cfg', 'Salesforce/T5_base_prefix_spider_with_cell_value.cfg', 
            '--output_dir', './tmp']
parser = HfArgumentParser((WrappedSeq2SeqTrainingArguments,))
training_args, = parser.parse_args_into_dataclasses()
set_seed(training_args.seed)
args = Configure.Get(training_args.cfg)

In [ ]:
# Load tokenizer and model(21->1 multitasked prefix)
tokenizer = AutoTokenizer.from_pretrained("hkunlp/from_all_T5_base_prefix_spider_with_cell_value2", use_fast=False)
from models.unified.prefixtuning import Model
model = Model(args)
model.load("hkunlp/from_all_T5_base_prefix_spider_with_cell_value2")

In [ ]:
struct_in = "| concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , average | singer : singer_id , name , country ( France ) , song_name , song_release_year , age , is_male | concert : concert_id , concert_name , theme , stadium_id , year | singer_in_concert : concert_id , singer_id"
text_in = "what is the minimum, average, and maximum age of all singers from France?"
# seq_out = "SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'"

In [ ]:
generated_sql = play("{}; structed knowledge: {}".format(text_in, struct_in), model, tokenizer)

In [ ]:
tokenized_txt = tokenizer([text_in], max_length=1024, padding="max_length", truncation=True)
print("tokenized_txt:", tokenized_txt)
print("tokenized_txt:", len(tokenized_txt['input_ids'][0]))
print("tokenized_txt:", len(tokenized_txt['attention_mask'][0]))

In [ ]:
generated_sql

In [ ]:
'''
Author: Ziyu Zhao
Affiliation: UWA NLT-TLP GROUP
'''

import os, re, math
from typing import Set

import sqlite3
import pandas as pd

class DBengine:
    
    """
    An Entity, which takes a sqlite3 database schema.
    Attributes:
    
    """

    def __init__(self, fdb):

        self.conn = sqlite3.connect(fdb)      
        self.conn.text_factory = lambda b: b.decode(errors = 'ignore')

    
    def execute(self, query):
      cursor = self.conn.cursor()  
      return cursor.execute(query)  
    
    def get_table_names(self):
        cursor = self.conn.cursor()  
        return cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")     
    
    def get_table_values(self, table_name):
        cursor = self.conn.cursor()  
        return cursor.execute('SELECT * from {}'.format(table_name))  

    def get_schema(self, table_name):
        cursor = self.conn.cursor()  
        cursor.execute('SELECT sql FROM sqlite_master WHERE type = "table" and name= "{}";'.format(table_name))  
        result = cursor.fetchone()
        if result is None:
            raise ValueError("Table %s does not exist" % table_name)
        return result[0].strip()  

    def get_outbound_foreign_keys(self, table_name):
        cursor = self.conn.cursor() 
        infos = cursor.execute("PRAGMA foreign_key_list([{}])".format(table_name)).fetchall()
        table_constraints = []
        pks_fks_dict ={}
        for info in infos:
            if info is not None:
                id, seq, ref_table, from_, to_, on_update, on_delete, match = info
                table_constraints.append(
                {"this_table": table_name, "column": from_, "ref_table": ref_table, "ref_column": to_}
                )
                pks_fks_dict[from_]= to_  # ref_column (aka pk in ref_table) is supposed to be the same with column. 
        return table_constraints, pks_fks_dict

    def get_primay_keys(self, table_name):
        cursor = self.conn.cursor() 
        pks = cursor.execute('SELECT l.name FROM pragma_table_info("{}") as l WHERE l.pk <> 0;'.format(table_name)).fetchall()
        return pks  
    
    def check_compound_pk(self, primary_keys):
        compound_pk_check=False
        if len(primary_keys)>1: 
            compound_pk_check=True
        return compound_pk_check

    def close(self):
        self.conn.cursor.close()

In [ ]:
db_path = '/home/22842219/Desktop/openSource/UnifiedSKGG-Cypher/data/downloads/spider/database/concert_singer/concert_singer.sqlite'
engine = DBengine(db_path)
for sql_query in generated_sql:
    print(sql_query)
    sql_ans = engine.execute(sql_query).fetchall()
    print("sql_ans:", sql_ans)

In [ ]:
!pip install py2neo

In [ ]:

from py2neo import Graph
import configparser

config = configparser.ConfigParser()
config.read('/home/22842219/Desktop/phd/SemanticParser4Graph/application/config.ini')
filenames = config["FILENAMES"]

neo4j_uri = filenames['neo4j_uri']
neo4j_user = filenames['neo4j_user']
neo4j_password = filenames['neo4j_password']
graph = Graph(neo4j_uri, auth = (neo4j_user, neo4j_password))

In [ ]:
sql2cypher = "MATCH (singer:`concert_singer.singer`) \
WHERE singer.Country = 'France' \
RETURN min(singer.Age),avg(singer.Age),max(singer.Age)"
cypher_res = graph.run(sql2cypher).data()
print(cypher_res)

# end-to-end translation from text-to-cypher

In [ ]:
########################## unifiedSKG learning process*******************
import logging
import os
import time

import torch
import datasets
import transformers
from transformers import (
    HfArgumentParser,
    set_seed,
    EarlyStoppingCallback,
)
from transformers.trainer_utils import get_last_checkpoint
from utils.configue import Configure
from utils.training_arguments import WrappedSeq2SeqTrainingArguments

from collections import OrderedDict
import utils.tool
from utils.dataset import TokenizedDataset
from utils.trainer import EvaluateFriendlySeq2SeqTrainer

# Huggingface realized the "Seq2seqTrainingArguments" which is the same with "WrappedSeq2SeqTrainingArguments"
# in transformers==4.10.1 during our work.
logger = logging.getLogger(__name__)



In [ ]:
args = Configure.Get(training_args.cfg)
print(training_args.cfg)
print(args.bert, args.bert.location)

In [ ]:
print(training_args)

In [ ]:
if 'checkpoint-???' in args.bert.location:
    args.bert.location = get_last_checkpoint(
        os.path.dirname(args.bert.location.model_name_or_path))
    logger.info(f"Resolve model_name_or_path to {args.bert.location.model_name_or_path}")


In [ ]:
last_checkpoint = None
if os.path.isdir(
        training_args.output_dir) and training_args.do_train and not training_args.overwrite_output_dir:
    last_checkpoint = get_last_checkpoint(training_args.output_dir)
    if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. "
            "Use --overwrite_output_dir to overcome."
        )
    elif last_checkpoint is not None and training_args.resume_from_checkpoint is None:
        logger.info(
            f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
            "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
        )

os.makedirs(training_args.output_dir, exist_ok=True)

In [ ]:
print(not args.arg_paths)
print(args.dataset.loader_path)  ## hey, attention!
print(args.dataset.data_store_path)
print(args.seq2seq.constructor)
print(args.arg_paths)

In [ ]:
# The inputs will be train, dev, test or train, dev now.
# We deprecate the k-fold cross-valid function 
# since it causes too many avoidable troubles.
if not args.arg_paths:
    cache_root = os.path.join('output', 'cache')
    os.makedirs(cache_root, exist_ok=True)
    raw_datasets_split: datasets.DatasetDict = datasets.load_dataset(path=args.dataset.loader_path,
                                                                        cache_dir=args.dataset.data_store_path)
    seq2seq_dataset_split: tuple = utils.tool.get_constructor(args.seq2seq.constructor)(args).to_seq2seq(
        raw_datasets_split, cache_root)


In [ ]:
cache_root = os.path.join('output', 'cache')
print(f'cache_root: {cache_root}')
os.makedirs(cache_root, exist_ok=True)
meta_tuning_data = {}


In [ ]:
for task, arg_path in args.arg_paths:
    task_args = Configure.Get(arg_path)
    task_args.bert = args.bert
    print(f'task: {task}, arg_path: {arg_path}, task_args: {task_args}')
    print('task_args.bert.location:', task_args.bert.location)
    print(f'task_args.dataset: {task_args.dataset}, task_args.dataset.loader_path: {task_args.dataset.loader_path}')
    print(f'task_args.dataset.data_store_path: {task_args.dataset.data_store_path}')
    task_raw_datasets_split: datasets.DatasetDict = datasets.load_dataset(
        path=task_args.dataset.loader_path,
        cache_dir=task_args.dataset.data_store_path)
    print(f'task_raw_datasets_split: {task_raw_datasets_split}')

    print(f'task_args.seq2seq.constructor: {task_args.seq2seq.constructor}')
    task_seq2seq_dataset_split: tuple = utils.tool.get_constructor(task_args.seq2seq.constructor)(task_args).\
        to_seq2seq(task_raw_datasets_split, cache_root)

    meta_tuning_data[arg_path] = task_seq2seq_dataset_split
    print(f'arg_path: {arg_path}')
    print(f'meta_tuning_data: {meta_tuning_data}')

seq2seq_dataset_split: tuple = utils.tool.get_constructor(args.seq2seq.constructor)(args).\
    to_seq2seq(meta_tuning_data)
print(f'seq2seq_dataset_split: {seq2seq_dataset_split}')

In [ ]:
print(task_args)
print(args)


In [ ]:
print(f'args: {args}, args.evaluate.tool: {args.evaluate.tool}')
evaluator = utils.tool.get_evaluator(args.evaluate.tool)(args)
print(f'evaluator: {evaluator}')

In [ ]:
print(f'args.model.name: {args.model.name}, args: {args}')
model = utils.tool.get_model(args.model.name)(args)

In [ ]:
model_tokenizer = model.tokenizer
model_tokenizer

In [ ]:
seq2seq_train_dataset, seq2seq_eval_dataset, seq2seq_test_dataset = None, None, None
print(f'seq2seq_train_dataset: {seq2seq_train_dataset}')

In [ ]:
print(f'len(seq2seq_dataset_split): {len(seq2seq_dataset_split)}')
if len(seq2seq_dataset_split) == 2:
    seq2seq_train_dataset, seq2seq_eval_dataset = seq2seq_dataset_split
elif len(seq2seq_dataset_split) == 3:
    seq2seq_train_dataset, seq2seq_eval_dataset, seq2seq_test_dataset = seq2seq_dataset_split
    print(f'seq2seq_train_dataset: {seq2seq_train_dataset}, seq2seq_eval_dataset: {seq2seq_eval_dataset} \
        seq2seq_test_dataset: {seq2seq_test_dataset}')
else:
    raise ValueError("Other split not support yet.")

In [ ]:
from utils.dataset import TokenizedDataset

In [ ]:
# We wrap the "string" seq2seq data into "tokenized tensor".
train_dataset = TokenizedDataset(args, training_args, model_tokenizer,
                                    seq2seq_train_dataset) if seq2seq_train_dataset else None
eval_dataset = TokenizedDataset(args, training_args, model_tokenizer,
                                seq2seq_eval_dataset) if seq2seq_eval_dataset else None
test_dataset = TokenizedDataset(args, training_args, model_tokenizer,
                                seq2seq_test_dataset) if seq2seq_test_dataset else None
print(f'train_dataset: {train_dataset}')
print(f'eval_dataset: {eval_dataset}')
print(f'test_dataset: {test_dataset}')


In [ ]:
# Initialize our Trainer
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=args.seq2seq.patience if args.seq2seq.patience else 5)
print(f'early_stopping_callback: {early_stopping_callback}')
print(f'patience: {args.seq2seq.patience}')

In [ ]:
from utils.trainer import EvaluateFriendlySeq2SeqTrainer


In [ ]:
print(f'seq2seq_eval_dataset: {seq2seq_eval_dataset}')

In [ ]:
!export WANDB_API_KEY=edd277e1d782e96634109705a6235577032da246
!export WANDB_PROJECT='text-to-sql(text2cypher-24Jan)'
!export WANDB_ENTITY=leamonzea929

In [ ]:
print(training_args.report_to)
print(training_args.local_rank <=0)
print("MLFLOW_EXPERIMENT_ID" in os.environ)
print(os.getenv("WANDB_PROJECT", "uni-frame-for-knowledge-tabular-tasks"))
print(training_args.run_name)
print(os.getenv("WANDB_ENTITY", 'sgtnew'))

In [ ]:
trainer = EvaluateFriendlySeq2SeqTrainer(
    args=training_args,
    model=model,
    evaluator=evaluator,
    # We name it "evaluator" while the hugging face call it "Metric",
    # they are all f(predictions: List, references: List of dict) = eval_result: dict
    tokenizer=model_tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    eval_examples=seq2seq_eval_dataset,
    # wandb_run_dir=wandb.run.dir if "wandb" in training_args.report_to and training_args.local_rank <= 0 else None,
    callbacks=[early_stopping_callback],
)
print('Trainer build successfully.')

In [ ]:
print(training_args.load_weights_from)

# Load model weights (for --do_train=False or post finetuning).
if training_args.load_weights_from:
    state_dict = torch.load(os.path.join(training_args.load_weights_from, transformers.WEIGHTS_NAME), map_location="cpu")
    trainer.model.load_state_dict(state_dict, strict=True)
    # release memory
    del state_dict

In [ ]:
print(args.load_multiple_prefix_module_weights_from)
if args.load_multiple_prefix_module_weights_from:
    reconstruct_state_dict = OrderedDict()

    # load prefix modules
    for task_name, module_weight_location in args.load_multiple_prefix_module_weights_from:
        state_dict = torch.load(os.path.join(module_weight_location, transformers.WEIGHTS_NAME), map_location="cpu")
        MULTI_PREFIX_ATTR_NAME = "multi_prefix"
        for weight_name, stored_tensor in state_dict.items():
            if str(weight_name).startswith("pretrain_model"):
                continue  # skip the pretrained model and we will load a new one from another place
            reconstruct_state_dict['{}.{}.{}'.format(MULTI_PREFIX_ATTR_NAME, "_".join(task_name.split("_")[:-1]), weight_name)] = stored_tensor
            # extract the prefix part and add them to dict

    # give it into the model
    trainer.model.load_state_dict(reconstruct_state_dict, strict=False)

    # release memory
    del reconstruct_state_dict